# import packages

In [ ]:
# setting base

import os

import warnings, sys, os ## system
if not sys.warnoptions:
    warnings.simplefilter("ignore") # ignore warnings
    
import mne

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import scipy.io

# load EEG signal file

In [ ]:
# load file

user_path = os.path.expanduser('~')
vhdr_path = os.path.join('_','sub-01_decoder-nback.vhdr')
# remove _, and enter your data path instead

raw = mne.io.read_raw_brainvision(vhdr_path,eog=('HEOG','HEOG1','VEOG'))
# data used in this project is collected by 'brainvision'

In [ ]:
# apply pass filter

raw.load_data().filter(0.5, 40) 
# import filter (high-pass, low-pass)
events = mne.events_from_annotations(raw) 
# define the trigger

event_id = {'object': 101, 'scene': 102}
# identify event triggers

In [ ]:
# re-reference signal

raw.plot()
raw.set_eeg_reference('average', projection=True)
raw.plot()

In [ ]:
def load_raw(var_name):
    
    # read raw file
    var_name = mne.io.read_raw_brainvision(vhdr_path,eog=('HEOG','HEOG1','VEOG'),
                                           preload=True)
    
    # set pass-filter
    var_name.load_data().filter(0.5, 40) 
    
    # apply re-reference using average signal
    var_name.set_eeg_reference('average', projection=True)
    
    return var_name

# control eye movement signal

## op1) reject blinked trial

In [ ]:
# use pure raw data

load_raw(REJ_raw)

In [ ]:
# find eye blink

eog_events = mne.preprocessing.find_eog_events(REJ_raw) 
# find blink signal
onsets = eog_events[:, 0] / REJ_raw.info['sfreq'] - 0.25
durations = [0.5] * len(eog_events)
descriptions = ['bad blink'] * len(eog_events)
blink_annot = mne.Annotations(onsets, durations, descriptions,
                              orig_time=REJ_raw.info['meas_date'])
REJ_raw.set_annotations(blink_annot)

In [ ]:
# pick blink and plot

eeg_picks = mne.pick_types(REJ_raw.info, eeg=True)
REJ_raw.plot(events=eog_events, order=eeg_picks)

## op2) ICA projection

In [ ]:
# use pure raw data

load_raw(ICA_raw)

In [ ]:
# summize noise from EOG

eog_evoked = create_eog_epochs(ICA_raw).average()
eog_evoked.apply_baseline(baseline=(None, -0.2))

In [ ]:
ica = ICA(n_components=15, max_iter='auto', random_state=97)
# use first 15 PCs in ICA
ica.fit(ICA_raw)

In [ ]:
# use EOG channel

eog_indices, eog_scores = ica.find_bads_eog(ICA_raw)
# find which ICs match the EOG pattern
ica.exclude = eog_indices

# barplot of ICA component "EOG match" scores
ica.plot_scores(eog_scores)

# plot ICs applied to raw data, with EOG matches highlighted
ica.plot_sources(ICA_raw, show_scrollbars=False)

# plot ICs applied to the averaged EOG epochs, with EOG matches highlighted
ica.plot_sources(eog_evoked)

In [ ]:
ica.exclude = [0]
# indices chosen based on various plots above

In [ ]:
# apply ICA exclusion

# ica.apply() changes the Raw object in-place
ica.apply(ICA_raw)
ICA_raw.plot()

## op3) SSP projection

In [ ]:
# use pure raw data

load_raw(SSP_raw)

In [ ]:
# make SSP projector

blink_proj = mne.preprocessing.compute_proj_eog(SSP_raw, tmax=1.5, n_grad=0, n_mag=0,
                                                ch_name=['HEOG','HEOG1','VEOG'])

In [ ]:
# add made SSP projector

SSP_raw.add_proj(blink_proj[1])

# epoching signal

In [ ]:
# Read epochs

tmin, tmax = -0.200, 1.500 # cut time-course
epochs = mne.Epochs(_, events[0], event_id, tmin, tmax, _,
                    picks='eeg', baseline=(-0.2, 0.0), preload=True,
                    reject=None, decim=3, verbose='error')
# remove first _, and enter variable name
# remove second _, and enter proj=True if you determined to use SSP

# decoding signal

In [ ]:
# save data in variable

X = epochs.get_data()  
# n_epochs, n_eeg_channels, n_times
y = epochs.events[:, 2]  
# target event

In [ ]:
# evaluate score using cross-validation : regression model

lin_pipe = make_pipeline(Scaler(train_epochs.info), 
                         Vectorizer(), LogisticRegression(solver='liblinear'))

lin_scores = cross_val_multiscore(lin_pipe, X, y, cv=5, n_jobs=None)
# cross-validation

lin_score = np.mean(lin_scores, axis=0)
# Mean scores across cross-validation splits
# mean cross-validation score from each split
#  -> scores == 1 * len(cv) array

print(lin_score)

In [ ]:
# evaluate score using cross-validation : support vector machine

svm_pipe = make_pipeline(Scaler(train_epochs.info), Vectorizer(), 
                         SVC(decision_function_shape='ovo'))

svm_scores = cross_val_multiscore(svm_pipe, X, y, cv=5, n_jobs=None)
# cross-validation

svm_score = np.mean(svm_scores, axis=0)
# Mean scores across cross-validation splits
# mean cross-validation score from each split
#  -> scores == 1 * len(cv) array

print(svm_score)